In [1]:
from instance_reader import read_instance
from heuristics import greedy_wave_selection
from Low_levels import *
import random

from funciones_auxiliares import seleccionar_segun_probabilidad

# instancia

In [32]:
instance = read_instance("datasets/a/instance_0004.txt")

# Low Levels

In [33]:
low_level1 = LowLevel1(id=1, nombre= "LowLevel1")
low_level2 = LowLevel2(id=2, nombre= "LowLevel2")
low_level3 = LowLevel3(id=3, nombre= "LowLevel3")
low_level4 = LowLevel4(id=4, nombre= "LowLevel4")
low_level5 = LowLevel5(id=5, nombre= "LowLevel5")
# Definimos una lista de niveles bajos

low_levels = [low_level1, low_level2,low_level3, low_level4, low_level5]

In [34]:
mejor_solucion = None
best_value = 0
epocs = 10000

solucion_inicial_1 = instance.constructora1()
solucion_inicial_2 = instance.constructora2()
P_ij = np.ones((len(low_levels), len(low_levels)))
T_ij = np.zeros((len(low_levels), len(low_levels)))

# inicializamos la matriz T_ij con P_ij / sum_{k in low_levels} P_ik
for i in range(len(low_levels)):
    suma_P_ik = sum(P_ij[i])
    for j in range(len(low_levels)):
        T_ij[i][j] = P_ij[i][j] / suma_P_ik if suma_P_ik > 0 else 0


matriz_secuencias = []
for _ in range(epocs):
    solucion = random.choice([solucion_inicial_2])
    mejor_solucion_temporal = solucion
    best_value_temporal = solucion.objective_value
    low_level_inicial = None
    secuencia = []

    while True:

        # si no hay una low level inicial, se elige una al azar
        if low_level_inicial is None:
            low_level_elegida = random.choice(low_levels)
            id_eleccion = low_level_elegida.id
        else:
            probabilidades_siguiente_low_level = T_ij[low_level_inicial.id - 1,:]
            id_eleccion = seleccionar_segun_probabilidad(probabilidades_siguiente_low_level)
            low_level_elegida = low_levels[id_eleccion]

        secuencia.append(id_eleccion)


        solucion_nueva = low_level_elegida.implementacion(solucion)

        if solucion_nueva.is_factible == False: #or solucion_nueva.objective_value <= solucion.objective_value:
            break

        if solucion_nueva.objective_value > best_value_temporal:
            mejor_solucion_temporal = solucion_nueva
            best_value_temporal = solucion_nueva.objective_value

        if low_level_inicial is None:
            pass
        else:
            if solucion_nueva.objective_value > best_value_temporal:
                P_ij[low_level_inicial.id - 1][low_level_elegida.id - 1] += 1

        if low_level_inicial is None:
            id_low_level_inicial = low_level_elegida.id
           # la matriz T_ij se actualiza con el valor de P_ij/ sum_{k in low_levels} P_ik
            suma_P_ik = sum(P_ij[id_low_level_inicial - 1])
            for low_level in low_levels:
                low_level_id = low_level.id
                T_ij[id_low_level_inicial - 1][low_level_id - 1] = P_ij[id_low_level_inicial - 1][low_level_id - 1] / suma_P_ik if suma_P_ik > 0 else 0
            
        low_level_inicial = low_level_elegida
        solucion = solucion_nueva
    
    if best_value_temporal > best_value:
        mejor_solucion = solucion
        best_value = best_value_temporal
        print(f"Nueva mejor solucion encontrada:, Con mejor valor{best_value}, secuencia: {secuencia}")
    matriz_secuencias.append(secuencia)

print("Mejor solucion final:", mejor_solucion)

Nueva mejor solucion encontrada:, Con mejor valor0.43902439024390244, secuencia: [2, 1, 2]
Nueva mejor solucion encontrada:, Con mejor valor0.4567901234567901, secuencia: [5, 4, 1, 3, 4]
Nueva mejor solucion encontrada:, Con mejor valor0.475, secuencia: [4, 3, 1, 0, 4]
Nueva mejor solucion encontrada:, Con mejor valor0.4878048780487805, secuencia: [1, 0, 4]
Nueva mejor solucion encontrada:, Con mejor valor0.49382716049382713, secuencia: [5, 1, 4, 1, 3, 0, 4, 4]
Nueva mejor solucion encontrada:, Con mejor valor0.5, secuencia: [5, 4, 4, 1, 1, 0, 3, 3, 0, 2]
Nueva mejor solucion encontrada:, Con mejor valor0.5063291139240507, secuencia: [4, 3, 4, 1, 0, 3, 0, 2]
Nueva mejor solucion encontrada:, Con mejor valor0.5128205128205128, secuencia: [2, 1, 0, 0, 3, 0, 0, 4, 3, 3, 3, 4]
Nueva mejor solucion encontrada:, Con mejor valor0.5194805194805194, secuencia: [1, 3, 3, 3, 3, 3, 0, 2]
Nueva mejor solucion encontrada:, Con mejor valor0.5333333333333333, secuencia: [4, 3, 1, 3, 1, 4, 3, 0, 3, 0, 

In [27]:
P_ij[0,:]

array([1., 1., 1., 1., 1.])